# Debug: Option 8 Problem Household Count Discrepancy

This notebook investigates why we're finding 13 problem households instead of the expected 165.

Possible causes:
1. Different threshold for identifying "problem" households
2. Different aggregation level (person vs tax_unit vs household)
3. Different definition of "problem" (absolute vs percentage difference)
4. Data or simulation changes

## Setup

In [1]:
from policyengine_us import Microsimulation
from src.reforms import get_option8_reform
import numpy as np
import pandas as pd

# Create Option 8 reform
reform = get_option8_reform()

# Create microsimulation
print("Loading microsimulation data...")
sim = Microsimulation(reform=reform)
print("Done!")

year = 2026

c:\Users\dtsax\envs\pe\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading microsimulation data...
Done!


## Test 1: Different Thresholds at Household Level

In [2]:
# Calculate at household level
social_security = sim.calculate("social_security", year, map_to="household")
taxable_social_security = sim.calculate("taxable_social_security", year, map_to="household")
household_weight = sim.calculate("household_weight", year)

ss_values = social_security.values if hasattr(social_security, 'values') else social_security
taxable_ss_values = taxable_social_security.values if hasattr(taxable_social_security, 'values') else taxable_social_security
weight_values = household_weight.values if hasattr(household_weight, 'values') else household_weight

# Test different thresholds
thresholds = [0.01, 0.10, 1.00, 10.00, 100.00, 1000.00]

print("Testing different absolute difference thresholds (household level):")
print("="*100)
for threshold in thresholds:
    problem_mask = (ss_values > 0) & (np.abs(taxable_ss_values - ss_values) > threshold)
    problem_indices = np.where(problem_mask)[0]
    
    unweighted_count = len(problem_indices)
    weighted_count = weight_values[problem_indices].sum() if len(problem_indices) > 0 else 0
    
    print(f"Threshold ${threshold:>8,.2f}: {unweighted_count:>6,} unweighted | {weighted_count:>12,.0f} weighted")

Testing different absolute difference thresholds (household level):
Threshold $    0.01:     13 unweighted |      121,336 weighted
Threshold $    0.10:     13 unweighted |      121,336 weighted
Threshold $    1.00:     13 unweighted |      121,336 weighted
Threshold $   10.00:     13 unweighted |      121,336 weighted
Threshold $  100.00:     13 unweighted |      121,336 weighted
Threshold $1,000.00:     13 unweighted |      121,336 weighted


## Test 2: Percentage-Based Thresholds

In [3]:
# Calculate percentage difference
pct_difference = np.divide(
    np.abs(taxable_ss_values - ss_values),
    ss_values,
    out=np.zeros_like(ss_values, dtype=float),
    where=ss_values > 0
)

pct_thresholds = [0.001, 0.01, 0.05, 0.10, 0.50, 0.90]

print("\nTesting different percentage difference thresholds:")
print("="*100)
for pct_threshold in pct_thresholds:
    problem_mask = (ss_values > 0) & (pct_difference > pct_threshold)
    problem_indices = np.where(problem_mask)[0]
    
    unweighted_count = len(problem_indices)
    weighted_count = weight_values[problem_indices].sum() if len(problem_indices) > 0 else 0
    
    print(f"Threshold {pct_threshold*100:>6.1f}%: {unweighted_count:>6,} unweighted | {weighted_count:>12,.0f} weighted")


Testing different percentage difference thresholds:
Threshold    0.1%:     13 unweighted |      121,336 weighted
Threshold    1.0%:     13 unweighted |      121,336 weighted
Threshold    5.0%:     13 unweighted |      121,336 weighted
Threshold   10.0%:     13 unweighted |      121,336 weighted
Threshold   50.0%:      9 unweighted |      118,202 weighted
Threshold   90.0%:      9 unweighted |      118,202 weighted


## Test 3: Check Different Aggregation Levels

In [4]:
# Check at person level
print("\nChecking different aggregation levels:")
print("="*100)

for map_level in ["person", "tax_unit", "household"]:
    ss = sim.calculate("social_security", year, map_to=map_level)
    taxable_ss = sim.calculate("taxable_social_security", year, map_to=map_level)
    
    ss_vals = ss.values if hasattr(ss, 'values') else ss
    taxable_ss_vals = taxable_ss.values if hasattr(taxable_ss, 'values') else taxable_ss
    
    # Use $0.01 threshold
    problem_mask = (ss_vals > 0) & (np.abs(taxable_ss_vals - ss_vals) > 0.01)
    problem_count = problem_mask.sum()
    
    print(f"{map_level:15s}: {problem_count:>6,} problem units with |taxable_ss - ss| > $0.01")


Checking different aggregation levels:
person         :    165 problem units with |taxable_ss - ss| > $0.01
tax_unit       :     14 problem units with |taxable_ss - ss| > $0.01
household      :     13 problem units with |taxable_ss - ss| > $0.01


## Test 4: Check for Less-Than-100% Taxation (Not Equal)

In [5]:
# Maybe the original analysis was looking for households where taxable_ss < ss (not equal)
# rather than taxable_ss != ss

print("\nComparing different problem definitions:")
print("="*100)

# Definition 1: Not equal (with tolerance)
mask1 = (ss_values > 0) & (np.abs(taxable_ss_values - ss_values) > 0.01)
count1_unweighted = mask1.sum()
count1_weighted = weight_values[mask1].sum()
print(f"Definition 1 - Not equal (|diff| > $0.01):     {count1_unweighted:>6,} unweighted | {count1_weighted:>12,.0f} weighted")

# Definition 2: Less than (undertaxed)
mask2 = (ss_values > 0) & ((ss_values - taxable_ss_values) > 0.01)
count2_unweighted = mask2.sum()
count2_weighted = weight_values[mask2].sum()
print(f"Definition 2 - Undertaxed (ss - taxable > $0.01): {count2_unweighted:>6,} unweighted | {count2_weighted:>12,.0f} weighted")

# Definition 3: Less than 99% taxed
ratio = np.divide(taxable_ss_values, ss_values, out=np.zeros_like(ss_values, dtype=float), where=ss_values > 0)
mask3 = (ss_values > 0) & (ratio < 0.99)
count3_unweighted = mask3.sum()
count3_weighted = weight_values[mask3].sum()
print(f"Definition 3 - Less than 99% taxed:            {count3_unweighted:>6,} unweighted | {count3_weighted:>12,.0f} weighted")

# Definition 4: Less than 100% taxed (ratio < 1.0)
mask4 = (ss_values > 0) & (ratio < 1.0)
count4_unweighted = mask4.sum()
count4_weighted = weight_values[mask4].sum()
print(f"Definition 4 - Less than 100% taxed (ratio<1):  {count4_unweighted:>6,} unweighted | {count4_weighted:>12,.0f} weighted")


Comparing different problem definitions:
Definition 1 - Not equal (|diff| > $0.01):         13 unweighted |      121,336 weighted
Definition 2 - Undertaxed (ss - taxable > $0.01):     13 unweighted |      121,336 weighted
Definition 3 - Less than 99% taxed:                13 unweighted |      121,336 weighted
Definition 4 - Less than 100% taxed (ratio<1):     497 unweighted |    2,074,012 weighted


## Test 5: Check Tax Unit Level with Different Thresholds

In [6]:
# Maybe the original analysis was at tax_unit level?
print("\nTesting tax_unit level with various thresholds:")
print("="*100)

ss_tu = sim.calculate("social_security", year, map_to="tax_unit")
taxable_ss_tu = sim.calculate("taxable_social_security", year, map_to="tax_unit")
tax_unit_weight = sim.calculate("tax_unit_weight", year)

ss_tu_vals = ss_tu.values if hasattr(ss_tu, 'values') else ss_tu
taxable_ss_tu_vals = taxable_ss_tu.values if hasattr(taxable_ss_tu, 'values') else taxable_ss_tu
tu_weight_vals = tax_unit_weight.values if hasattr(tax_unit_weight, 'values') else tax_unit_weight

thresholds = [0.01, 0.10, 1.00, 10.00, 100.00]

for threshold in thresholds:
    problem_mask = (ss_tu_vals > 0) & (np.abs(taxable_ss_tu_vals - ss_tu_vals) > threshold)
    
    unweighted_count = problem_mask.sum()
    weighted_count = tu_weight_vals[problem_mask].sum() if unweighted_count > 0 else 0
    
    print(f"Threshold ${threshold:>8,.2f}: {unweighted_count:>6,} unweighted | {weighted_count:>12,.0f} weighted")


Testing tax_unit level with various thresholds:
Threshold $    0.01:     14 unweighted |      121,336 weighted
Threshold $    0.10:     14 unweighted |      121,336 weighted
Threshold $    1.00:     14 unweighted |      121,336 weighted
Threshold $   10.00:     14 unweighted |      121,336 weighted
Threshold $  100.00:     14 unweighted |      121,336 weighted


## Test 6: Distribution Analysis

In [7]:
# Let's look at the distribution of differences
print("\nDistribution of absolute differences (household level):")
print("="*100)

has_ss = ss_values > 0
abs_diff = np.abs(ss_values - taxable_ss_values)

# Count households by difference ranges
ranges = [
    (0, 0.01, "$0 - $0.01 (essentially equal)"),
    (0.01, 1, "$0.01 - $1"),
    (1, 10, "$1 - $10"),
    (10, 100, "$10 - $100"),
    (100, 1000, "$100 - $1,000"),
    (1000, 10000, "$1,000 - $10,000"),
    (10000, float('inf'), "$10,000+")
]

for min_val, max_val, label in ranges:
    mask = has_ss & (abs_diff >= min_val) & (abs_diff < max_val)
    unweighted = mask.sum()
    weighted = weight_values[mask].sum() if unweighted > 0 else 0
    
    if unweighted > 0:
        print(f"{label:35s}: {unweighted:>8,} unweighted | {weighted:>15,.0f} weighted")


Distribution of absolute differences (household level):
$0 - $0.01 (essentially equal)     :    6,930 unweighted |      51,624,936 weighted
$1,000 - $10,000                   :        7 unweighted |          17,930 weighted
$10,000+                           :        6 unweighted |         103,407 weighted


## Test 7: Summary Comparison Table

In [8]:
# Create a summary showing which definition might match the expected 165
print("\n" + "="*100)
print("SUMMARY: Which definition gives ~165 unweighted households?")
print("="*100)

results = []

# Test combinations
for level, (ss_data, taxable_data, weight_data) in [
    ("household", (ss_values, taxable_ss_values, weight_values)),
    ("tax_unit", (ss_tu_vals, taxable_ss_tu_vals, tu_weight_vals))
]:
    for threshold in [0.01, 0.10, 1.00, 10.00]:
        mask = (ss_data > 0) & (np.abs(taxable_data - ss_data) > threshold)
        unweighted = mask.sum()
        weighted = weight_data[mask].sum() if unweighted > 0 else 0
        
        results.append({
            'Level': level,
            'Threshold': f'${threshold:.2f}',
            'Unweighted': unweighted,
            'Weighted': int(weighted),
            'Match?': '*** MATCH ***' if 160 <= unweighted <= 170 else ''
        })

df = pd.DataFrame(results)
print(df.to_string(index=False))

print("\n" + "="*100)
matches = df[df['Match?'] != '']
if len(matches) > 0:
    print(f"Found {len(matches)} configuration(s) that match ~165 households!")
else:
    print("No configuration found that matches ~165 households.")
    print("The data or reform implementation may have changed.")
print("="*100)


SUMMARY: Which definition gives ~165 unweighted households?
    Level Threshold  Unweighted  Weighted Match?
household     $0.01          13    121336       
household     $0.10          13    121336       
household     $1.00          13    121336       
household    $10.00          13    121336       
 tax_unit     $0.01          14    121336       
 tax_unit     $0.10          14    121336       
 tax_unit     $1.00          14    121336       
 tax_unit    $10.00          14    121336       

No configuration found that matches ~165 households.
The data or reform implementation may have changed.
